In [ ]:
import pygame
import random
from PIL import Image

# --- Configuration ---
WIDTH, HEIGHT = 1200, 720
FPS = 30
SECONDS = 16  # Duration of the loop
TOTAL_FRAMES = FPS * SECONDS
GIF_FILENAME = "matrix_seamless_loop.gif"

# Colors
BLACK = (0, 0, 0)
GREEN_BRIGHT = (150, 255, 150) # Almost white-green for the leading character
GREEN_NORMAL = (0, 255, 70)    # Standard matrix green
GREEN_DARK = (0, 50, 0)        # Deep background green

# --- Matrix Characters (Katakana + Numbers) ---
# Generating the iconic unicode characters
KATAKANA = [chr(i) for i in range(0x30A0, 0x30FF)] 
NUMBERS = [chr(i) for i in range(0x30, 0x39)]
SYMBOLS = KATAKANA + NUMBERS

class MatrixStream:
    def __init__(self, x, font_size, speed_multiplier, brightness_alpha):
        self.x = x
        self.font_size = font_size
        # Try multiple fonts that support Unicode - pygame will use first available
        try:
            self.font = pygame.font.SysFont('notosanscjkjp,notosansjp,hiraginosans,mspgothic,yugothic,arial', font_size, bold=True)
        except:
            # Fallback to default font if none of the above work
            self.font = pygame.font.Font(None, font_size)
        
        self.brightness_alpha = brightness_alpha
        
        # --- SEAMLESS LOOP MATH ---
        # To loop perfectly, the stream must travel a distance that allows it to 
        # return to exactly the same relative position by the end of the GIF.
        # We set the "Loop Height" to be larger than the screen so streams don't pop in/out visibly.
        self.loop_height = HEIGHT + (font_size * 15) # Buffer for the tail
        
        # Speed = Distance / Time. 
        # We multiply by an integer (speed_multiplier) to allow some columns to lap 
        # the screen twice or three times in one loop cycle, creating speed variation.
        self.speed = (self.loop_height * speed_multiplier) / TOTAL_FRAMES
        
        # Random starting offset (vertical), snapped to grid to look clean
        self.start_y = random.randrange(0, int(self.loop_height), font_size)
        
        # The content of the stream (the vertical string of characters)
        self.stream_len = random.randint(5, 15)
        self.chars = [random.choice(SYMBOLS) for _ in range(self.stream_len)]
        
        # Pre-render chars to improve performance
        self.rendered_chars = []
        for i, char in enumerate(self.chars):
            # Head is bright, tail fades out
            if i == 0: 
                color = GREEN_BRIGHT
            elif i < 3:
                color = GREEN_NORMAL
            else:
                # Fade color towards dark green
                factor = 1 - (i / self.stream_len)
                color = (0, int(255 * factor), int(70 * factor))
                
            surf = self.font.render(char, True, color)
            
            # Apply Transparency (Perspective depth)
            if brightness_alpha < 255:
                surf.set_alpha(brightness_alpha)
                
            self.rendered_chars.append(surf)

    def draw(self, surface, frame_index):
        # Calculate current Y position based on frame index
        # formula: (Start + Speed * Time) % Modulo_Distance
        current_y = (self.start_y + self.speed * frame_index) % self.loop_height
        
        # We draw the stream. Since we are using modulo, we handle the "wrap around"
        # by checking if the stream is near the bottom and drawing a copy at the top
        # to ensure it looks continuous.
        
        for i, char_surf in enumerate(self.rendered_chars):
            # Position of this specific character in the stream
            char_y = current_y - (i * self.font_size)
            
            # Draw primary instance
            if -self.font_size < char_y < HEIGHT:
                surface.blit(char_surf, (self.x, char_y))
            
            # Draw wrap-around instance (if head is low, tail might be entering top, 
            # or if head is high, tail might be leaving bottom - simplified here)
            # If the y is near the bottom, draw a copy shifted up by loop_height
            if char_y > HEIGHT - (self.font_size * 20):
                 surface.blit(char_surf, (self.x, char_y - self.loop_height))
            
            # If the y is near the top (negative), draw a copy shifted down
            if char_y < 0:
                 surface.blit(char_surf, (self.x, char_y + self.loop_height))


def generate_matrix_loop():
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    
    # --- Create Layers for Perspective ---
    streams = []
    
    # Layer 1: Background (Darker, Smaller, Slower)
    # They loop 1 time per GIF duration
    font_size = 14
    cols = WIDTH // font_size
    for i in range(cols):
        streams.append(MatrixStream(
            x=i * font_size, 
            font_size=font_size, 
            speed_multiplier=1, # Slow
            brightness_alpha=80
        ))

    # Layer 2: Midground
    # They loop 2 times per GIF duration
    font_size = 22
    cols = WIDTH // font_size
    for i in range(cols):
        # Randomly skip columns to make it less dense
        if random.random() > 0.4:
            streams.append(MatrixStream(
                x=i * font_size, 
                font_size=font_size, 
                speed_multiplier=2, # Medium speed
                brightness_alpha=180
            ))

    # Layer 3: Foreground (Bright, Big, Fast)
    # They loop 2 or 3 times per GIF duration
    font_size = 35
    cols = WIDTH // font_size
    for i in range(cols):
        if random.random() > 0.7: # Sparse foreground
            streams.append(MatrixStream(
                x=i * font_size, 
                font_size=font_size, 
                speed_multiplier=3, # Fast
                brightness_alpha=255
            ))

    frames = []
    print(f"Generating {TOTAL_FRAMES} frames for a seamless loop...")

    # --- Render Loop ---
    # We do NOT use a while loop here. We iterate exactly through the frame count.
    for f in range(TOTAL_FRAMES):
        # Handle OS events to keep window responsive (optional)
        pygame.event.pump() 

        # Clear screen every frame (No trails needed, the 'stream' object handles the tail)
        screen.fill(BLACK)

        # Draw all streams
        for stream in streams:
            stream.draw(screen, f)

        pygame.display.flip()

        # Capture
        raw_str = pygame.image.tostring(screen, "RGB", False)
        image = Image.frombytes("RGB", (WIDTH, HEIGHT), raw_str)
        frames.append(image)

        if f % 10 == 0:
            print(f"Rendered frame {f}/{TOTAL_FRAMES}")

    pygame.quit()

    # --- Save ---
    print("Saving GIF...")
    frames[0].save(
        GIF_FILENAME,
        save_all=True,
        append_images=frames[1:],
        optimize=False,
        duration=1000 // FPS,
        loop=0 # Infinite loop
    )
    print(f"Done! {GIF_FILENAME} created.")

if __name__ == "__main__":
    generate_matrix_loop()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html
Generating 480 frames for a seamless loop...
Rendered frame 0/480
Rendered frame 10/480
Generating 480 frames for a seamless loop...
Rendered frame 0/480
Rendered frame 10/480


2025-12-13 00:38:52.481 python[40869:4128218] +[IMKClient subclass]: chose IMKClient_Modern
2025-12-13 00:38:52.481 python[40869:4128218] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Rendered frame 20/480
Rendered frame 30/480
Rendered frame 40/480
Rendered frame 50/480
Rendered frame 40/480
Rendered frame 50/480
Rendered frame 60/480
Rendered frame 70/480
Rendered frame 60/480
Rendered frame 70/480
Rendered frame 80/480
Rendered frame 90/480
Rendered frame 100/480
Rendered frame 80/480
Rendered frame 90/480
Rendered frame 100/480
Rendered frame 110/480
Rendered frame 120/480
Rendered frame 130/480
Rendered frame 110/480
Rendered frame 120/480
Rendered frame 130/480
Rendered frame 140/480
Rendered frame 150/480
Rendered frame 140/480
Rendered frame 150/480
Rendered frame 160/480
Rendered frame 170/480
Rendered frame 160/480
Rendered frame 170/480
Rendered frame 180/480
Rendered frame 190/480
Rendered frame 200/480
Rendered frame 180/480
Rendered frame 190/480
Rendered frame 200/480
Rendered frame 210/480
Rendered frame 220/480
Rendered frame 230/480
Rendered frame 210/480
Rendered frame 220/480
Rendered frame 230/480
Rendered frame 240/480
Rendered frame 250/480
Re

: 